In [1]:
import omnipath as op
import pandas as pd

from collections import defaultdict
import networkx as nx

In [8]:
ot_drugs = pd.read_csv("../data/opentargets_knowndrugs.tsv", sep="\t")
ot_disease = pd.read_csv("../data/opentargets_disease_associations.tsv", sep="\t")

In [9]:
ot_disease

,targetId,diseaseId,datatypeId,datatypeHarmonicScore,datatypeEvidenceCount,diseaseLabel,targetName,targetSymbol
0,ENSG00000000003,EFO_0000305,rna_expression,0.017417,1,breast carcinoma,tetraspanin 6,TSPAN6
1,ENSG00000000003,EFO_0000311,literature,0.020000,1,cancer,tetraspanin 6,TSPAN6
2,ENSG00000000005,EFO_0000305,rna_expression,0.025026,2,breast carcinoma,tenomodulin,TNMD
3,ENSG00000000005,EFO_0000311,literature,0.004000,1,cancer,tenomodulin,TNMD
4,ENSG00000000419,EFO_0000311,literature,0.019600,2,cancer,dolichyl-phosphate mannosyltransferase subunit...,DPM1
...,...,...,...,...,...,...,...,...
2653072,ENSG00000282728,EFO_0000313,literature,0.009129,6,carcinoma,PGR antisense RNA 1,PGR-AS1
2653073,ENSG00000284190,EFO_0000313,literature,0.006000,1,carcinoma,microRNA 21,MIR21
2653074,ENSG00000285219,EFO_0000313,literature,0.011600,2,carcinoma,hepatocellular carcinoma up-regulated long non...,HULC
2653075,ENSG00000285972,EFO_0000313,literature,0.004000,1,carcinoma,competing endogenous lncRNA 2 for microRNA let-7b,CERNA2


In [14]:
kg = nx.DiGraph()

In [15]:
for ri, row in ot_disease.iterrows():
    gene = row["targetSymbol"]
    disease_id = row["diseaseId"]
    disease_label = row["diseaseLabel"]
    
    if not gene in kg.nodes:
        kg.add_node(gene, type="gene")
        
    if not disease_id in kg.nodes:
        kg.add_node(disease_id, type="disease", name=disease_label)

In [16]:
print(kg)

DiGraph with 41768 nodes and 0 edges


In [18]:
for ri, row in ot_drugs.iterrows():
    drug = row["drugId"]
    disease_id = row["diseaseId"]
    gene = row["targetGeneSymbol"]
    
    if not gene in kg.nodes:
        kg.add_node(gene, type="gene")
        
    if not drug in kg.nodes:
        kg.add_node(drug, type="drug")
        
    if not disease_id in kg.nodes:
        kg.add_node(disease_id, type="disease")

In [19]:
print(kg)

DiGraph with 44981 nodes and 0 edges


In [21]:

for ri, row in ot_disease.iterrows():
        
    gene = row["targetSymbol"]
    disease_id = row["diseaseId"]
    
    disease_score = row["datatypeHarmonicScore"]
    disease_evidences = row["datatypeEvidenceCount"]
    disease_evidence_source = row["datatypeId"]
    
    disease_data = {
        "disease_score": disease_score,
        "disease_evidences": disease_evidences,
        "disease_evidence_source": disease_evidence_source,
        "type": "interacts"
    }
    
    kg.add_edge( disease_id, gene, **disease_data )

In [22]:
print(kg)

DiGraph with 44981 nodes and 2532829 edges


In [23]:
ot_drugs

,drugId,targetId,diseaseId,status,diseaseName,targetGeneSymbol,targetGeneName
0,CHEMBL628,ENSG00000113231,DOID_13406,Completed,pulmonary sarcoidosis,PDE8B,phosphodiesterase 8B
1,CHEMBL1096882,ENSG00000175482,EFO_0000095,"Active, not recruiting",chronic lymphocytic leukemia,POLD4,"DNA polymerase delta 4, accessory subunit"
2,CHEMBL1096882,ENSG00000177084,EFO_0000095,Completed,chronic lymphocytic leukemia,POLE,"DNA polymerase epsilon, catalytic subunit"
3,CHEMBL1096882,ENSG00000100479,EFO_0000095,Recruiting,chronic lymphocytic leukemia,POLE2,"DNA polymerase epsilon 2, accessory subunit"
4,CHEMBL1096882,ENSG00000101868,EFO_0000095,Unknown status,chronic lymphocytic leukemia,POLA1,"DNA polymerase alpha 1, catalytic subunit"
...,...,...,...,...,...,...,...
224257,CHEMBL744,ENSG00000168356,Orphanet_94147,Unknown status,Spinocerebellar ataxia type 7,SCN11A,sodium voltage-gated channel alpha subunit 11
224258,CHEMBL284348,ENSG00000158445,Orphanet_97,Withdrawn,Familial paroxysmal ataxia,KCNB1,potassium voltage-gated channel subfamily B me...
224259,CHEMBL1431,ENSG00000189043,Orphanet_98473,Completed,Muscular dystrophy,NDUFA4,NDUFA4 mitochondrial complex associated
224260,CHEMBL1431,ENSG00000198840,Orphanet_98473,Completed,Muscular dystrophy,MT-ND3,mitochondrially encoded NADH:ubiquinone oxidor...


In [24]:

for ri, row in ot_drugs.iterrows():
    #drugId	targetId	diseaseId	status	diseaseName	targetGeneSymbol	targetGeneName

        
    drug = row["drugId"]
    disease_id = row["diseaseId"]
    drug_target = row["targetGeneSymbol"]
    
    evidence_status = row["status"]
    
    drug_disease_data = {
        "evidence_status": evidence_status
    }
    
    kg.add_edge( drug, disease_id, **drug_disease_data )
    kg.add_edge( drug, drug_target, type="targeted_by" )

In [25]:
print(kg)

DiGraph with 44981 nodes and 2576959 edges
